In [1]:
# import requests
import pandas as pd
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from time import sleep
from lxml import etree, html

In [2]:
# Tests include biochemistry, hematology, coagulation test, out-patient
tests = ["BC", "HE", "CT", "OP"]
# Open Chrome browser
browser = webdriver.Chrome("./chromedriver")

for test in tests:
    url = "https://www.ntuh.gov.tw/labmed/檢驗目錄/Lists/2015/{}.aspx".format(test)
    roughtable = dict(test_ename = [], test_cname = [], specimen = [], reference_range = [], detail_link = [])
    # Make the browser visit the page
    browser.get(url)
    # Attempt to turn the page 10 times
    for i in range(10):
        # Transform what browser gets into an element tree
        root = etree.fromstring(browser.page_source, etree.HTMLParser())
        # Parse the information with Xpath
        for row in root.xpath("//table[@class='ms-listviewtable']/tbody/tr"):
            roughtable["test_ename"].append(row.xpath("./td/div/a/text()")[0].strip())
            roughtable["test_cname"].append(row.xpath("./td/text()")[0].strip())
            try:
                roughtable["specimen"].append(row.xpath("./td/div/font/text()")[0].strip())
            except IndexError:
                roughtable["specimen"].append(row.xpath("./td/div/div/text()")[0].strip())
            roughtable["reference_range"].append("\n".join(map(lambda x: x.strip(),row.xpath("./td/div/font/text()")[1:])))
            roughtable["detail_link"].append(row.xpath("./td/div/a/@href")[0].strip())
        try:
            browser.find_element_by_xpath("//td[@id='pagingWPQ2next']").click()
            sleep(5)
        except NoSuchElementException:
            # Write the csv and break the for loop if there is no next page
            pd.DataFrame(roughtable)[["test_ename", "test_cname", "specimen", "reference_range", "detail_link"]].to_csv("{}.csv".format(test))
            break
browser.quit()

In [3]:
# # Get the source code of the html with requests
# # This method can NOT retrieve the following pages of dynamic webpage
# result = ""
# while result == "":
#     try:
#         # Certificate is not verified to bypass the SSLError
#         # Not secure though
#         result = requests.get(url, verify = False)
#         break
#     except:
#         sleep(5)
#         continue
# # Transform it into an element tree
# root = etree.fromstring(result.content, etree.HTMLParser())
# # Parse the information with Xpath
# # The Xpath for root is somehow different
# for row in root.xpath("//table[@class='ms-listviewtable']/tr"):
#     tests["test_ename"].append(row.xpath("./td/div/a/text()")[0].strip())
#     tests["test_cname"].append(row.xpath("./td/text()")[0].strip())
#     try:
#         tests["specimen"].append(row.xpath("./td/div/font/text()")[0].strip())
#     except IndexError:
#         tests["specimen"].append(row.xpath("./td/div/div/text()")[0].strip())
#     tests["reference_range"].append("\n".join(map(lambda x: x.strip(),row.xpath("./td/div/font/text()")[1:])))
#     tests["detail_link"].append(row.xpath("./td/div/a/@href")[0].strip())